In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("after_day1.csv")
df.drop(columns=["Unnamed: 0"], inplace=True)

In [4]:
df["BurnoutRisk"] = df["BurnoutRisk"].astype(int)

In [5]:
df.drop(["EmployeeID"], axis= 1, inplace= True)

In [6]:
numerical_cols = [col for col in df.columns if df[col].dtype != 'object' and col != "BurnoutRisk"]
categorical_cols = [col for col in df.columns if df[col].dtype == 'object']

In [7]:
from sklearn.preprocessing import PowerTransformer

scaler = PowerTransformer()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

In [8]:
df['RemoteWork'] = df['RemoteWork'].map({'No': 1, 'Hybrid': 2, 'Yes': 3})
df['HasMentalHealthSupport'] = df['HasMentalHealthSupport'].map({'No': 1, 'Yes': 2})
df['HasTherapyAccess'] = df['HasTherapyAccess'].map({'No': 1, 'Yes': 2})
df['SalaryRange'] = df['SalaryRange'].map({'<40K': 1, '40K-60K': 2, '60K-80K': 3, '80K-100K': 4, '100K+': 5})

In [ ]:
from sklearn.preprocessing import OneHotEncoder

nominal_cols = list(set(categorical_cols) - set(['RemoteWork', 'HasMentalHealthSupport', 'HasTherapyAccess', 'SalaryRange']))
if nominal_cols:
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    encoded_nominals = encoder.fit_transform(df[nominal_cols])
    encoded_nominal_df = pd.DataFrame(
        encoded_nominals,
        columns=encoder.get_feature_names_out(nominal_cols),
        index=df.index
    )

df = df.drop(columns=nominal_cols)
df = pd.concat([df, encoded_nominal_df], axis=1)

In [13]:
X = df.drop(columns=["BurnoutRisk"])
y = df["BurnoutRisk"]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)
y_pred_lr_binary = (y_pred_lr >= 0.5).astype(int) 

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

linear = LinearRegression()
linear.fit(X_train, y_train)

y_pred = linear.predict(X_test)
print("MSE: ", mean_squared_error(y_test, y_pred))
print("R2: ", r2_score(y_test, y_pred))

MSE:  0.08359930723905563
R2:  0.6247365851686427


In [17]:
df.to_csv('after_day2.csv')